# Async `Prompto` vs. Python for loop

In [21]:
from model_utils import send_gemini, send_openai, send_ollama, send_prompt
from dataset_utils import load_prompt_dicts

In [22]:
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

True

In [23]:
def send_prompts(prompt_dicts: list[dict]) -> list[str]:
    # maive for loop to synchronously dispatch prompts
    for prompt_dict in prompt_dicts:
        send_prompt(prompt_dict)

In [24]:
prompt_dicts = load_prompt_dicts("./data/input/openai-gpt-3.5.jsonl")

In [25]:
prompt_dicts

[{'id': 0,
  'api': 'openai',
  'model_name': 'gpt-3.5-turbo',
  'prompt': 'How does technology impact us?',
  'parameters': {'n': 1, 'temperature': 1, 'max_tokens': 100}},
 {'id': 0,
  'api': 'openai',
  'model_name': 'gpt-3.5-turbo',
  'prompt': 'What is the North Pole?',
  'parameters': {'n': 1, 'temperature': 1, 'max_tokens': 100}},
 {'id': 0,
  'api': 'openai',
  'model_name': 'gpt-3.5-turbo',
  'prompt': 'What is the capital of France?',
  'parameters': {'n': 1, 'temperature': 1, 'max_tokens': 100}}]

In [27]:
send_prompts(prompt_dicts)

# Small tests

In [ ]:
ollama_test = send_ollama(prompt="Hello, world!", model_name="llama3", params={})

In [ ]:
gemini_test = send_gemini(
    prompt="Hello, world!", model_name="gemini-1.5-flash", params={}
)

In [ ]:
openai_test = send_openai(prompt="Hello, world!", model_name="gpt-3.5-turbo", params={})